In [ ]:
import requests
import pdf2image

url = "https://www.criticallink.com/wp-content/uploads/ISO-9001-2015-Certificate.pdf"
response = requests.get(url, timeout=30)
images = pdf2image.convert_from_bytes(response.content)

In [ ]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_docx
# load the pdf file to drive
# split the file to text, table and images
def doc_partition(path,file_name):
  raw_pdf_elements = partition_pdf(
    filename=path + file_name,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path="../data/images/",
    )

  return raw_pdf_elements
path = "../data/images/"
file_name = "LLaVA.pdf"
raw_pdf_elements = doc_partition(path,file_name)

In [ ]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.docx import partition_docx

# Path to save 
path = "../data/docx/"

# Get elements
raw_pdf_elements = partition_docx(
    filename=path + "these.docx",
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    # Hard max on chunks
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
)

In [ ]:
raw_pdf_elements

In [ ]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

In [ ]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

In [ ]:
text_elements